# Educational Spendings Effect on Home Valuations Across the United States of America


In [29]:
# import libraries 

import pandas as pd
import numpy as np
import statsmodels.api as sm

# Read datasets

gdpDeflator = pd.read_csv('data\FRED_ipd.csv')

homeValuation = pd.read_csv('data\ZillowHousingValues_state-by-state.csv')

educationSpending = pd.read_csv('data\education-spending_state-by-state.csv')
homeValuation.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31
0,9,0,California,state,NaN,"186,276.11","186,903.75","187,750.03","189,586.30","191,719.03",...,"770,917.50","771,314.83","766,685.74","759,238.76","752,637.61","747,513.15","742,356.81","735,996.33","730,505.21","728,133.50"
1,54,1,Texas,state,NaN,"105,214.36","105,270.24","105,296.59","105,431.58","105,518.34",...,"296,589.89","299,379.18","300,228.91","299,532.99","298,589.12","297,623.44","296,121.32","294,864.99","294,073.12","294,336.35"
2,14,2,Florida,state,NaN,"103,672.98","103,897.88","104,170.16","104,723.38","105,319.56",...,"380,194.36","385,446.11","388,077.61","388,365.45","387,892.62","387,242.03","385,828.83","384,293.21","383,029.02","383,063.09"
3,43,3,New York,state,NaN,"123,754.51","124,195.87","124,620.23","125,532.44","126,474.47",...,"409,537.86","412,732.59","413,850.47","412,716.52","411,295.16","410,125.74","408,268.09","407,584.88","408,242.02","411,304.03"
4,47,4,Pennsylvania,state,NaN,"89,306.93","89,498.08","89,677.69","90,043.63","90,419.51",...,"240,680.87","241,661.58","241,588.25","240,795.85","240,652.06","240,982.62","241,276.58","241,860.23","242,563.19","243,858.90"


 Format gdpDeflator to be used homeValuations and educationSpending. This table will eventually be used to merge and perform gdp deflation calculations.

In [30]:
# Rename columns
gdpDeflator = gdpDeflator.rename(columns={'DATE': 'year', 'USAGDPDEFAISMEI': 'ipd'})

# Set date to correct variable type
gdpDeflator['year'] = pd.to_datetime(gdpDeflator['year']) 

# remove all data that does not fit data date range
gdpDeflator = gdpDeflator[gdpDeflator['year'] >= '2000-01-01']
gdpDeflator = gdpDeflator[gdpDeflator['year'] <= '2016-01-01']

# reset index to accommodate new datarange
gdpDeflator = gdpDeflator.reset_index(drop=True)
print(gdpDeflator)



         year    ipd
0  2000-01-01  74.53
1  2001-01-01  76.21
2  2002-01-01  77.40
3  2003-01-01  78.92
4  2004-01-01  81.04
5  2005-01-01  83.58
6  2006-01-01  86.16
7  2007-01-01  88.49
8  2008-01-01  90.19
9  2009-01-01  90.77
10 2010-01-01  91.86
11 2011-01-01  93.77
12 2012-01-01  95.52
13 2013-01-01  97.19
14 2014-01-01  99.01
15 2015-01-01 100.00
16 2016-01-01 101.00


Adjust Base year from 2015 to 2016

In [31]:
# set base year
base_year = '2016-01-01'

# calculate conversion factor
conversion_factor = gdpDeflator.loc[gdpDeflator['year'] == base_year, 'ipd'].values[0] / 100

# adjust to base year of 2016
gdpDeflator['ipd'] = gdpDeflator['ipd'] / conversion_factor

print(gdpDeflator)

         year    ipd
0  2000-01-01  73.79
1  2001-01-01  75.45
2  2002-01-01  76.63
3  2003-01-01  78.14
4  2004-01-01  80.24
5  2005-01-01  82.75
6  2006-01-01  85.31
7  2007-01-01  87.61
8  2008-01-01  89.29
9  2009-01-01  89.87
10 2010-01-01  90.95
11 2011-01-01  92.84
12 2012-01-01  94.57
13 2013-01-01  96.23
14 2014-01-01  98.03
15 2015-01-01  99.01
16 2016-01-01 100.00


Format educationSpending to be merged with gdpDeflator (for inflation calculations)

In [32]:
# convert wide form dataframe into long form
educationSpending = pd.melt(educationSpending, id_vars=['state'], var_name='year', value_name='USD')

# change date variable type
educationSpending['year'] = pd.to_datetime(educationSpending['year'])

# sort by state, then year for easier indexing
educationSpending = educationSpending.sort_values(['state', 'year'])

# remove all data that does not fit data date range
educationSpending = educationSpending[educationSpending['year'] >= '2000-01-01']
educationSpending = educationSpending[educationSpending['year'] <= '2016-01-01']

# reset index after sort
educationSpending = educationSpending.reset_index(drop=True)

# rename columns
educationSpending = educationSpending.rename(columns={'USD': 'EducationSpend'})

print(educationSpending)

       state       year  EducationSpend
0    Alabama 2000-01-01         4006894
1    Alabama 2001-01-01         4140053
2    Alabama 2002-01-01         4273211
3    Alabama 2003-01-01         4429789
4    Alabama 2004-01-01         4586366
..       ...        ...             ...
862  Wyoming 2012-01-01         1418805
863  Wyoming 2013-01-01         1425713
864  Wyoming 2014-01-01         1453975
865  Wyoming 2015-01-01         1495720
866  Wyoming 2016-01-01         1542763

[867 rows x 3 columns]


Format homeValuation Dataframe

In [33]:
# drop unneccessary columns
collumnToDrop = ['RegionID', 'SizeRank', 'RegionType', 'StateName']
homeValuation = homeValuation.drop(collumnToDrop, axis=1)
# Convert wide form into long form dataframe
homeValuation = pd.melt(homeValuation, id_vars=['RegionName'], var_name='Date', value_name='HomeValue')

# convert ['Date'] datatype to DateTime
homeValuation['Date'] = pd.to_datetime(homeValuation['Date']) + pd.tseries.offsets.MonthBegin(-1)

# sort data by state and date
homeValuation = homeValuation.sort_values(['RegionName', 'Date'])

# Remove all data that does is not the start of a year
homeValuation = homeValuation[homeValuation['Date'].dt.month == 1]

# remove all data that does not fit data date range
homeValuation = homeValuation[homeValuation['Date'] <= '2016-01-01']
homeValuation = homeValuation[homeValuation['Date'] >= '2000-01-01']

# rename columns (allow for simple use of left join later)
homeValuation = homeValuation.rename(columns={'Date': 'year', 'RegionName': 'state'})

# reset index
homeValuation = homeValuation.reset_index(drop=True)
homeValuation[homeValuation.isna().any(axis=1)]

,state,year,HomeValue
442,Montana,2000-01-01,NaN
443,Montana,2001-01-01,NaN
444,Montana,2002-01-01,NaN
445,Montana,2003-01-01,NaN
446,Montana,2004-01-01,NaN
447,Montana,2005-01-01,NaN
527,New Mexico,2000-01-01,NaN
528,New Mexico,2001-01-01,NaN
529,New Mexico,2002-01-01,NaN
578,North Dakota,2000-01-01,NaN


Left join educationSpending and gdpDeflator, to be used in gdp deflation calculations

In [34]:
education_gdpDeflator = pd.merge(educationSpending, gdpDeflator, on='year', how='left')
education_gdpDeflator.head(20)

,state,year,EducationSpend,ipd
0,Alabama,2000-01-01,4006894,73.79
1,Alabama,2001-01-01,4140053,75.45
2,Alabama,2002-01-01,4273211,76.63
3,Alabama,2003-01-01,4429789,78.14
4,Alabama,2004-01-01,4586366,80.24
5,Alabama,2005-01-01,4925201,82.75
6,Alabama,2006-01-01,5478034,85.31
7,Alabama,2007-01-01,6104470,87.61
8,Alabama,2008-01-01,6671430,89.29
9,Alabama,2009-01-01,6629154,89.87


In [35]:
# define variable for base year value
BaseYear = gdpDeflator.loc[16, 'ipd'] / 100
StartYear = gdpDeflator.loc[0, 'ipd'] / 100
print(StartYear)
cummulative = ((BaseYear / StartYear) - 1) * 100
print(cummulative)

0.7378954563178582
35.520552598339464


In [36]:
#make new column
education_gdpDeflator['adjusted_EducationSpend'] = 0

#group data for use in for loop
education_gdpDeflator_group = education_gdpDeflator.groupby(['state', 'year'])

for info, group in education_gdpDeflator_group:
    state, year = info
    gdp_deflator = group['ipd'].iloc[0] / 100


In [37]:
pd.options.display.float_format = '{:,.2f}'.format
education_gdpDeflator.head(20)

,state,year,EducationSpend,ipd,adjusted_EducationSpend
0,Alabama,2000-01-01,4006894,73.79,0
1,Alabama,2001-01-01,4140053,75.45,0
2,Alabama,2002-01-01,4273211,76.63,0
3,Alabama,2003-01-01,4429789,78.14,0
4,Alabama,2004-01-01,4586366,80.24,0
5,Alabama,2005-01-01,4925201,82.75,0
6,Alabama,2006-01-01,5478034,85.31,0
7,Alabama,2007-01-01,6104470,87.61,0
8,Alabama,2008-01-01,6671430,89.29,0
9,Alabama,2009-01-01,6629154,89.87,0
